In [26]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [3]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [ ]:
torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomRotation(15),
                        transforms.RandomAffine(degrees=0, translate=(0.15, 0.15)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.131,), (0.308,)) # mean 0.13101533792088266
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.131,), (0.308,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [6]:
# Data to plot accuracy and loss graphs
train_losses = []
test_losses = []
train_acc = []
test_acc = []
train_batch_losses = []
train_batch_acc = []
test_batch_losses = []
test_batch_acc = []

def GetCorrectPredCount(pPrediction, pLabels):
  return pPrediction.argmax(dim=1).eq(pLabels).sum().item()

In [28]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, criterion):
    model.train()
    pbar = tqdm(train_loader)
    train_loss = 0
    correct = 0
    processed = 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        #loss = F.nll_loss(output, target)
        loss = criterion(output, target)
        train_loss+=loss.item()
        train_batch_losses.append(loss.item()) # Store batch loss
        loss.backward()
        optimizer.step()
        correct += GetCorrectPredCount(output, target)
        processed += len(data)
        Accuracy = float(100*correct/processed)
        #pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx} Accuracy={Accuracy}')
        train_batch_acc.append(Accuracy) # Store batch accuracy
        pbar.set_description(desc= f'loss={train_loss/len(train_loader)} batch_id={batch_idx} Accuracy={Accuracy}')
    train_acc.append(Accuracy)
    train_losses.append(train_loss/len(train_loader))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            #test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            loss = criterion(output, target).item()
            test_loss += loss
            test_batch_losses.append(loss)
            #pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            #correct += pred.eq(target.view_as(pred)).sum().item()
            correct += GetCorrectPredCount(output, target)
            test_batch_acc.append(100. * correct / len(test_loader.dataset)) # Store batch accuracy

    test_loss /= len(test_loader.dataset)
    test_acc.append(100. * correct / len(test_loader.dataset))
    test_losses.append(test_loss)
    Accuracy = float(float(100. * correct) / len(test_loader.dataset))

    print('\nTest set: Average loss: {:.2f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        Accuracy))

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 8, 3, padding=1, bias=False),
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.Dropout(0.05),
            nn.Conv2d(8, 16, 3, padding=1, bias=False),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Dropout(0.05),
            nn.MaxPool2d(2,2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, 3, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout(0.05),
            nn.Conv2d(32, 40, 3, padding=1, bias=False),
            nn.BatchNorm2d(40),
            nn.ReLU(),
            nn.Dropout(0.05),
            nn.MaxPool2d(2,2)
        )
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(40, 10)
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.gap(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [32]:
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              72
       BatchNorm2d-2            [-1, 8, 28, 28]              16
              ReLU-3            [-1, 8, 28, 28]               0
            Conv2d-4           [-1, 16, 28, 28]           1,152
       BatchNorm2d-5           [-1, 16, 28, 28]              32
              ReLU-6           [-1, 16, 28, 28]               0
         MaxPool2d-7           [-1, 16, 14, 14]               0
            Conv2d-8           [-1, 32, 14, 14]           4,608
       BatchNorm2d-9           [-1, 32, 14, 14]              64
             ReLU-10           [-1, 32, 14, 14]               0
           Conv2d-11           [-1, 40, 14, 14]          11,520
      BatchNorm2d-12           [-1, 40, 14, 14]              80
             ReLU-13           [-1, 40, 14, 14]               0
        MaxPool2d-14             [-1, 4

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=15)
scheduler.state_dict()

{'step_size': 10,
 'gamma': 0.2,
 'base_lrs': [0.001],
 'last_epoch': 0,
 '_step_count': 1,
 '_is_initial': False,
 '_get_lr_called_within_step': False,
 '_last_lr': [0.001]}

In [34]:
num_epochs = 15
for epoch in range(1, num_epochs+1):
    print(f'Epoch {epoch}')
    train(model, device, train_loader, optimizer, criterion)
    test(model, device, test_loader)
    scheduler.step()

Epoch 1


loss=0.8898803702990215 batch_id=1874 Accuracy=90.16833333333334: 100%|██████████| 1875/1875 [02:36<00:00, 11.97it/s]




Test set: Average loss: 0.02, Accuracy: 9742/10000 (97.42%)

Epoch 2


loss=0.6469080656051636 batch_id=1874 Accuracy=97.35666666666667: 100%|██████████| 1875/1875 [02:37<00:00, 11.91it/s] 




Test set: Average loss: 0.02, Accuracy: 9860/10000 (98.60%)

Epoch 3


...


Test set: Average loss: 0.02, Accuracy: 9932/10000 (99.32%)

Epoch 14


loss=0.5717437333106995 batch_id=1874 Accuracy=98.91: 100%|██████████| 1875/1875 [03:51<00:00,  8.09it/s]             




Test set: Average loss: 0.02, Accuracy: 9935/10000 (99.35%)

Epoch 15


loss=0.5712053905169169 batch_id=1874 Accuracy=98.955: 100%|██████████| 1875/1875 [02:53<00:00, 10.78it/s]            




Test set: Average loss: 0.02, Accuracy: 9937/10000 (99.37%)



In [35]:
num_epochs = 5
for epoch in range(1, num_epochs+1):
    print(f'Epoch {epoch}')
    train(model, device, train_loader, optimizer, criterion)
    test(model, device, test_loader)
    scheduler.step()

Epoch 1


loss=0.570358858648936 batch_id=1874 Accuracy=98.96: 100%|██████████| 1875/1875 [03:07<00:00,  9.98it/s]              




Test set: Average loss: 0.02, Accuracy: 9932/10000 (99.32%)

Epoch 2


loss=0.5701137790044148 batch_id=1874 Accuracy=98.965: 100%|██████████| 1875/1875 [03:57<00:00,  7.90it/s]            




Test set: Average loss: 0.02, Accuracy: 9934/10000 (99.34%)

Epoch 3


...


Test set: Average loss: 0.02, Accuracy: 9935/10000 (99.35%)

Epoch 4


loss=0.5691644907633464 batch_id=1874 Accuracy=98.945: 100%|██████████| 1875/1875 [04:30<00:00,  6.93it/s]            




Test set: Average loss: 0.02, Accuracy: 9930/10000 (99.30%)

Epoch 5


loss=0.568643266805013 batch_id=1874 Accuracy=98.985: 100%|██████████| 1875/1875 [04:05<00:00,  7.64it/s]             




Test set: Average loss: 0.02, Accuracy: 9941/10000 (99.41%)

